# DEAD POET

This project aims to generate Shakeshpere style pargraphs using GANS. The GAN will try to generate a random pargraph which can later be improved to take some basic requirements like story line in the input itself to generate paragraph with those features.

###  Preparing The Dataset
1. Get the NLTk corpus
2. Join the Sentences to make a string
3. Combine Multiple Sentences to Make a Pragraph fo desired length.

In [ ]:
import pickle as pk
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import nltk.corpus as corpus
g_sents=corpus.gutenberg.sents()
g_sents=[" ".join(i) for i in g_sents]
data=[".".join(g_sents[i:i+25]) for i in range(len(g_sents)-1)]

### Preprocessing the Dataset

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
MAX_SEQUENCE_LENGTH=500
with open('word_index.pickle','rb') as word_index_file:
    word_index=pk.load(word_index_file)
tokenizer=Tokenizer(lower=True,filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.num_words=10000
tokenizer.fit_on_texts([" ".join(list(word_index.keys()))])
sequences=tokenizer.texts_to_sequences(data)
word_index=tokenizer.word_index
data=pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)

### Build Generator

In [2]:
import tensorflow as tf
import numpy as np
BATCH_SIZE=10
NOISE_SHAPE=[16,16]
init = tf.global_variables_initializer()
def generateRandomShape(length=256,batch_size=5000):
    
    inp=np.random.rand(batch_size,length)
    return inp

def build_generator(inp,batch_size):
    inp=tf.reshape(inp,shape=[batch_size,NOISE_SHAPE[0],NOISE_SHAPE[1],1])
    
    #conv2d layer 1
    g_l1=tf.layers.conv2d(inp,filters=16,kernel_size=(1,1),strides=(1,1),padding="SAME",name="g_l1")
    g_up1=tf.image.resize_images(g_l1,size=(50,18),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    
    #conv2d layer 2
    g_l2=tf.layers.conv2d(g_up1,filters=32,kernel_size=(2,2),strides=(1,1),padding="SAME",name="g_l2")
    g_up2=tf.image.resize_images(g_l2,size=(100,20),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    
    #conv2d layer 3
    g_l3=tf.layers.conv2d(g_up2,filters=16,kernel_size=(3,3),strides=(1,1),padding="SAME",name="g_l3")
    g_up3=tf.image.resize_images(g_l3,size=(250,25),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    
    #conv2d layer 4
    g_l4=tf.layers.conv2d(g_up3,filters=1,kernel_size=(3,3),strides=(1,1),padding="SAME",name="g_l4")
    g_up4=tf.image.resize_images(g_l4,size=(500,50),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    
    return g_up4

train_x=generateRandomShape(batch_size=BATCH_SIZE)
x_=tf.placeholder(dtype=tf.float32,shape=(None,256))
a=build_generator(x_,batch_size=BATCH_SIZE)

with tf.Session() as sess:
    init=tf.global_variables_initializer()
    tf.get_variable_scope().reuse_variables()
    sess.run(init)
    print(sess.run(a,{x_:train_x}).shape)


(10, 500, 50, 1)


### Build Descriminator

In [5]:
def build_descriminator():
    #conv1
    inps=tf.placeholder(dtype=tf.float64,shape=(None,None,50,1))
    d_=tf.layers.conv2d(inps,filters=32,kernel_size=(3,3),strides=1,name="d_l1")
    d_=tf.max_pooling2d(d_,pool_size=(2,2),name="d_mp1")
    d_=tf.layers.conv2d(inps,filters=16,kernel_size=(3,3),strides=1,name="d_l1")
    d_=tf.max_pooling2d(d_,pool_size=(2,2),name="d_mp1")